In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop

In [9]:
train = ImageDataGenerator(rescale=1/255)
validation = ImageDataGenerator(rescale=1/255)
testing = ImageDataGenerator(rescale=1/255)
batch_size = 64

In [10]:
train_dataset = train.flow_from_directory("/content/drive/MyDrive/tf_workspace/training_cnn/images/training",target_size=(200,200),batch_size=batch_size)
validation_dataset = validation.flow_from_directory("/content/drive/MyDrive/tf_workspace/training_cnn/images/validation",target_size=(200,200),batch_size=batch_size)
testing_dataset = testing.flow_from_directory("/content/drive/MyDrive/tf_workspace/training_cnn/images/testing",target_size=(200,200),batch_size=batch_size, shuffle=False)

Found 5310 images belonging to 4 classes.
Found 1325 images belonging to 4 classes.
Found 0 images belonging to 4 classes.


In [5]:
train_dataset.class_indices

{'bee': 0, 'bumbee': 1, 'insect': 2, 'vespula': 3}

In [6]:
def convolutional_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same', strides = (2,2))(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Processing Residue with conv(1,1)
    x_skip = tf.keras.layers.Conv2D(filter, (1,1), strides = (2,2))(x_skip)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x
def identity_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x
def ResNet34(shape = (32, 32, 3), classes = 10):
    # Step 1 (Setup Input Layer)
    x_input = tf.keras.layers.Input(shape)
    x = tf.keras.layers.ZeroPadding2D((3, 3))(x_input)
    # Step 2 (Initial Conv layer along with maxPool)
    x = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    # Define size of sub-blocks and initial filter size
    block_layers = [3, 4, 6, 3]
    filter_size = 64
    # Step 3 Add the Resnet Blocks
    for i in range(4):
        if i == 0:
            # For sub-block 1 Residual/Convolutional block not needed
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            # One Residual/Convolutional Block followed by Identity blocks
            # The filter size will go on increasing by a factor of 2
            filter_size = filter_size*2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)
    # Step 4 End Dense Network
    x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation = 'relu')(x)
    x = tf.keras.layers.Dense(classes, activation = 'softmax')(x)
    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
    return model
def customModel():
  return tf.keras.models.Sequential([tf.keras.layers.Conv2D(32,(3,3),activation = "relu", input_shape =(200,200,3)),
                              #Preprocessing
                              tf.keras.layers.RandomContrast(factor=0.1, seed=None),
                              tf.keras.layers.RandomRotation(factor=1,fill_mode='reflect',interpolation='bilinear',seed=None,fill_value=0.0),
                              tf.keras.layers.RandomFlip(),
                              #Training
                              tf.keras.layers.MaxPool2D(3,3),
                              tf.keras.layers.Conv2D(32,(3,3),activation = "relu"),
                              tf.keras.layers.MaxPool2D(3,3),
                              tf.keras.layers.Conv2D(64,(3,3),activation = "relu"),
                              tf.keras.layers.MaxPool2D(3,3),
                              tf.keras.layers.Conv2D(128,(3,3),activation = "relu"),
                              tf.keras.layers.MaxPool2D(3,3),
                              tf.keras.layers.Flatten(),
                              tf.keras.layers.Dense(512, activation = "relu"),
                              tf.keras.layers.Dense(4, activation = "softmax"),
                              ])

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')
tf.device('/device:GPU:0')
model = tf.keras.models.Sequential([
    # Input layer
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3), padding='same'),
    
    # Preprocessing
    tf.keras.layers.RandomRotation(factor=1, fill_mode='reflect', interpolation='bilinear', seed=None, fill_value=0.0),
    tf.keras.layers.RandomFlip(),
    
    # Feature extraction
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    
    # Classifier
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4, activation='softmax')
])
print(type(model))

<class 'keras.engine.sequential.Sequential'>


In [12]:
model =ResNet34(shape = (200, 200, 3), classes = 4)
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "/content/drive/MyDrive/tf_workspace/CNN_Test_5000_Custom/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
# Create a callback that saves the model's weights every 5 epochs
cp_callback = [
    tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq='epoch'),
    #tf.keras.callbacks.EarlyStopping(monitor='val_binary_crossentropy', patience=200),
]
# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))
model.compile(optimizer='adam', metrics=[tf.keras.metrics.CategoricalCrossentropy(name='categorical_crossentropy'),"accuracy"], loss=tf.keras.losses.CategoricalCrossentropy())

In [ ]:
with tf.device('/device:GPU:0'):
  model_fit = model.fit(train_dataset, callbacks=[cp_callback], epochs = 1000, validation_data=validation_dataset)

Epoch 1/1000
83/83 [==============================] - ETA: 0s - loss: 1.1641 - categorical_crossentropy: 1.1641 - accuracy: 0.5527
Epoch 1: saving model to /content/drive/MyDrive/tf_workspace/CNN_Test_5000_Custom/cp-0001.ckpt
83/83 [==============================] - 42s 501ms/step - loss: 1.1641 - categorical_crossentropy: 1.1641 - accuracy: 0.5527 - val_loss: 1.2501 - val_categorical_crossentropy: 1.2501 - val_accuracy: 0.4030
Epoch 2/1000
83/83 [==============================] - ETA: 0s - loss: 0.8906 - categorical_crossentropy: 0.8906 - accuracy: 0.6492
Epoch 2: saving model to /content/drive/MyDrive/tf_workspace/CNN_Test_5000_Custom/cp-0002.ckpt
83/83 [==============================] - 41s 497ms/step - loss: 0.8906 - categorical_crossentropy: 0.8906 - accuracy: 0.6492 - val_loss: 1.5988 - val_categorical_crossentropy: 1.5988 - val_accuracy: 0.3185
Epoch 3/1000
83/83 [==============================] - ETA: 0s - loss: 0.7791 - categorical_crossentropy: 0.7791 - accuracy: 0.6934
Epoch

In [ ]:
validation_dataset.class_indices

{'bee': 0, 'bumbee': 1, 'insect': 2, 'vespula': 3}

In [ ]:
from tensorflow.keras.preprocessing import image
from skimage import transform
validation_dataset.class_indices
dir_path = "/content/drive/MyDrive/tf_workspace/training_cnn/images/validation"
detWrong =[]
classes = 4
detWrongElement = []
for count in range(classes):
  detWrongElement.append([0,0,0,0,0])
for i in os.listdir(dir_path):
  for files in os.listdir(os.path.join(dir_path, i)):
    active_dir = os.path.join(dir_path, i)
    imagePath = os.path.join(active_dir, files)
    img = image.load_img(imagePath, target_size=(200,200))
    X = image.img_to_array(img)
    X = X.reshape(1, 200, 200, 3)
    #X = np.expand_dims(X,axis=0)
    #images = np.vstack([X])
    images = X.astype("float32")
    val = model.predict(images)
    detectionList = val.tolist()
    filename, _ = os.path.splitext(files)
    fileType = filename.split('(')
    fileTypeValue = None
    if(fileType[0] == "bee"):
      fileTypeValue = 0
    if(fileType[0] == "bumbeBee" or fileType[0] == "bumbleBee"):
      fileTypeValue = 1
    if(fileType[0] == "insect"):
      fileTypeValue = 2
    if(fileType[0] == "vespula"):
      fileTypeValue = 3
    if(fileTypeValue != None and detectionList[0][fileTypeValue] < 0.8):
      detWrong.append([detectionList[0],files])
      detWrongElement[fileTypeValue][0] +=1
      for index, element in enumerate(detectionList[0]):
        if(element >= 0.8):
          print(f"{filename} was identified as {index} with value {detectionList[0][fileTypeValue]}")
          print(detectionList)
          detWrongElement[fileTypeValue][index+1] +=1
    elif(fileTypeValue == None):
      print(filename)
print("bee, bumblebee, insect, vespula")
print(detWrongElement)

In [ ]:
# Validation set evaluation
val_loss, val_accuracy = model.evaluate(testing_dataset,
                                        steps=int(val_size/bs)+1)
print('Accuracy: {}'.format(val_accuracy))

# Validation set predict

y_val = testing_dataset.classes

pred = model.predict(testing_dataset,
                     verbose=1
                     steps=int(val_size/bs)+1)

accuracy_TTA = np.mean(np.equal(y_val, np.argmax(pred, axis=-1)))
print('Accuracy: {}'.format(accuracy_TTA))

In [ ]:
y_val = testing_dataset.classes
pred = model.predict(testing_dataset)
accuracy_TTA = np.mean(np.equal(y_val, np.argmax(pred, axis=-1)))
print('Accuracy: {}'.format(accuracy_TTA))

31/31 [==============================] - 54s 2s/step
Accuracy: 0.6597510373443983


In [ ]:
print(model.evaluate(testing_dataset, batch_size=128))

31/31 [==============================] - 1s 25ms/step - loss: 0.9511 - binary_crossentropy: 0.4151 - accuracy: 0.6598
[0.9510741233825684, 0.41510045528411865, 0.6597510576248169]


In [ ]:
from PIL import Image
import numpy as np
from skimage import transform
def load(filename):
   np_image = Image.open(filename)
   np_image = np.array(np_image).astype('float32')/255
   np_image = transform.resize(np_image, (200, 200, 3))
   np_image = np.expand_dims(np_image, axis=0)
   return np_image

image = load('/content/drive/MyDrive/tf_workspace/training_cnn/images/validation/bee/bee(1082).jpg')
print(model.predict(image))

1/1 [==============================] - 0s 319ms/step
[[9.4968331e-01 1.8599863e-03 4.8275307e-02 1.8138914e-04]]


In [ ]:
from tensorflow.keras.preprocessing import image
from PIL import Image
import numpy as np
from skimage import transform
def load(filename):
   np_image = Image.open(filename)
   np_image = np.array(np_image).astype('float32')/255
   np_image = transform.resize(np_image, (200, 200, 3))
   np_image = np.expand_dims(np_image, axis=0)
   return np_image
validation_dataset.class_indices
dir_path = "/content/drive/MyDrive/tf_workspace/training_cnn/images/validation"
detWrong =[]
classes = 4
detWrongAmount = 0
detRightAmount = 0
testedImages = 0
detWrongElement = []
for count in range(classes):
  detWrongElement.append([0,0,0,0,0])
for i in os.listdir(dir_path):
  for files in os.listdir(os.path.join(dir_path, i)):
    testedImages += 1
    active_dir = os.path.join(dir_path, i)
    imagePath = os.path.join(active_dir, files)
    images = load(imagePath)
    val = model.predict(images)
    detectionList = val.tolist()
    filename, _ = os.path.splitext(files)
    fileType = filename.split('(')
    fileTypeValue = None
    if(fileType[0] == "bee"):
      fileTypeValue = 0
    if(fileType[0] == "bumbeBee" or fileType[0] == "bumbleBee"):
      fileTypeValue = 1
    if(fileType[0] == "insect"):
      fileTypeValue = 2
    if(fileType[0] == "vespula"):
      fileTypeValue = 3
    if(fileTypeValue != None and detectionList[0][fileTypeValue] < 0.5):
      detWrongAmount += 1
      detWrong.append([detectionList[0],files])
      detWrongElement[fileTypeValue][0] +=1
      for index, element in enumerate(detectionList[0]):
        if(element >= 0.5):
          detWrongElement[fileTypeValue][index+1] +=1
    elif(fileTypeValue != None): detRightAmount += 1
    elif(fileTypeValue == None):
      print(filename)
print("bee, bumblebee, insect, vespula")
print(detWrongElement)
print(f"{testedImages} davon {detWrongAmount} falsch und {detRightAmount} richtig. Genauigkeit ist {detRightAmount / testedImages}")

1/1 [==============================] - 0s 36ms/step
bee, bumblebee, insect, vespula
[[58, 0, 3, 41, 2], [53, 9, 0, 25, 0], [33, 13, 1, 0, 9], [13, 0, 0, 11, 0]]
486 davon 157 falsch und 329 richtig. Genauigkeit ist 0.676954732510288


### Get number of files within the folder

In [ ]:
dirPath = "/content/drive/MyDrive/tf_workspace/training_cnn/images/validation"
os.listdir(dirPath)
for elements in os.listdir(dirPath):
  print(f"{elements}: {len(os.listdir(os.path.join(dirPath, elements)))}")

bee: 126
vespula: 32
insect: 237
bumbee: 91


# Load Model from Checkpoint

In [ ]:
# Loads the weights
cpt_path = "/content/drive/MyDrive/tf_workspace/CNN_Test_5000_Custom/cp-5000.ckpt"
model.load_weights(cpt_path)

# Re-evaluate the model
print(model.evaluate(testing_dataset))
#print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

31/31 [==============================] - 1s 28ms/step - loss: 0.9511 - binary_crossentropy: 0.4151 - accuracy: 0.6598
[0.9510740637779236, 0.41510042548179626, 0.6597510576248169]


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('/content/drive/MyDrive/exported_models/CNN_Test_5000_Custom/model.tflite', 'wb+') as f:
  f.write(tflite_model)

In [ ]:
from tensorflow.keras.preprocessing import image
from PIL import Image
import numpy as np
from skimage import transform

def load(filename):
   np_image = Image.open(filename)
   np_image = np.array(np_image).astype('float32')/255
   np_image = transform.resize(np_image, (200, 200, 3))
   np_image = np.expand_dims(np_image, axis=0)
   return np_image

interpreter = tf.lite.Interpreter(model_path="/content/drive/MyDrive/exported_models/CNN_Test_5000_Custom/model.tflite")
imagePath = "/content/drive/MyDrive/tf_workspace/training_cnn/images/training/bee/bee(1004).jpg"

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
interpreter.allocate_tensors()
    
# lectura y procesamiento de image
new_img = images = load(imagePath)
    
# input_details[0]['index'] = the index which accepts the input
print(input_details[0])
print(new_img)
interpreter.set_tensor(tensor_index=input_details[0]['index'],value=[new_img])
    
# realizar la prediccion del interprete
interpreter.invoke()
    
# output_details[0]['index'] = the index which provides the input
output_data = interpreter.get_tensor(output_details[0]['index'])

print(output_data)

{'name': 'serving_default_conv2d_input:0', 'index': 0, 'shape': array([  1, 200, 200,   3], dtype=int32), 'shape_signature': array([ -1, 200, 200,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
[[[[0.76817656 0.63852954 0.26108828]
   [0.7637547  0.6331174  0.2605587 ]
   [0.7553037  0.6225684  0.25961778]
   ...
   [0.7159212  0.5686961  0.09412665]
   [0.7205201  0.57598996 0.09925453]
   [0.7203822  0.5799511  0.09840209]]

  [[0.7697939  0.6431566  0.25863713]
   [0.76419675 0.63617736 0.25732398]
   [0.75379485 0.62317723 0.2556075 ]
   ...
   [0.72243303 0.56317693 0.08703258]
   [0.72493005 0.57269704 0.0964619 ]
   [0.7224416  0.5782252  0.09756826]]

  [[0.7676075  0.6369015  0.26195076]
   [0.76359916 0.63204074 0.26169685]
   [0.7558346  0.6223542  0.26102883]
   ...
   [0.71363    0.5706

ValueError: ignored

#TensorBoards

In [ ]:
%load_ext tensorboard